# Recommending apps which can be profitable on the play store and the app store

The goal of this notebook is to come up with suggestions on what apps to build which can be profitable on the play store and the app store. 

Before begining the project, the assumption is that this recommnendation would be for only free apps - apps which do not need any money to be downloaded and where the primary source of revenue is via ads. Greater the number of users who see and engage with the ads, the better. Hence the notebook would help the users and developers understand what type of apps can attract more users.

## Exploring the data

Since there are millions of app on both these platform (2 million iOS app and 2.1 million Android app as of Aug 2018,  [Source](https://www.statista.com/statistics/276623/number-of-apps-available-in-leading-app-stores/)), rather than spending time and money to collect data about all of them, we would do our study on any relevant publicly avaiable data set. 

Below are two examples of such datasets
* [~10k android apps collected in Aug, 2018](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv)
* [~7k iOs apps collected in July 2017](https://dq-content.s3.amazonaws.com/350/AppleStore.csv)

For our research, this data would suffice.

Let us see how the data looks.

In [2]:
# Read the data
from csv import reader

# Reading iOS Data
iosData = open("AppleStore.csv", encoding="utf8")
iosDataReader = reader(iosData)
iosApps = list(iosDataReader)

# Reading android Data
androidData = open("googleplaystore.csv", encoding="utf8")
androidDataReader = reader(androidData)
androidApps = list(androidDataReader)

In [3]:
# Exploring the dataset
def explore_data(dataset, start, end, rows_and_columns=False):
    '''
        Function to explore a dataset in the form of list of lists
        
        Parameters
            dataset - list of list
            start - start index of the data set which we want to see
            end - end index of the data set
            rows_and_columns - if this is True, then the function will
            print the total number of rows and columns as well; defualt
            is False
            
        Returns
            None
    '''
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row, "\n")
    
    if rows_and_columns:
        print("Number of rows : ", len(dataset))
        print("Number of columns : ", len(dataset[0]))

In [4]:
# Let us see the the first 3 rows in iOS data and also want to check
# the total number of rows and columns 
explore_data(iosApps, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

Number of rows :  7198
Number of columns :  16


Looking at the above output, we can see the first row is actually the header or the meta data. Let us save that in a separate headers list.

In [5]:
iosHeaders = iosApps[0]
print(iosHeaders)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


So there are 7197 (7198 - 1 as 1st row is the header) rows and
16 columns or properites of the data.  Let us do the same for android data now.

In [6]:
explore_data(androidApps, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up'] 

Number of rows :  10842
Number of columns :  13


In [7]:
androidHeaders = androidApps[0]
print(androidHeaders)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


While the column names are quite descriptive for Android, they may not be so for iOS. Feel free to go through the documentation for [ios](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps), as well as [android](https://www.kaggle.com/lava18/google-play-store-apps) data, to know more.

In [8]:
print(iosHeaders)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Looking at both the headers, we can get a good idea about the type of data. The fields which seem import for iOS are 
* currency
* price
* rating_count_tot
* user_rating
* prime_genre
* cont_rating

The fields which seem important for Android are
* Category
* Rating
* Reviews
* Installs
* Type
* Price
* Content Rating
* Genres


## Cleaning the data - Deleting wrong entry

Before trying to analyse the data let us
* Delete inaccurate data - correct or remove it
* Delete duplicate data - remove duplicates

Recall that we need to analyse only free apps and for an english only speaking audience. So
* Remove non english apps
* Remove paid apps

Also, if you went through the [documentation](https://www.kaggle.com/lava18/google-play-store-apps/discussion), you would notice a dedicated section in the Google Play dataset in which one [discussion describes an error](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015).

In [9]:
print(androidHeaders, len(androidHeaders))
for androidApp in androidApps: 
    if len(androidApp) != len(androidHeaders):
        print(androidApp, len(androidApp))
        print(androidApps.index(androidApp))

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 13
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up'] 12
10473


So we can see here that for app 10472 (10473-1 as 1st is the header) there are just 12 columns. Let us delete this row. We can see that Caetgory is missing. An alternate solution can be to quickly check the Google Play Store and find the cagtegory and add here but for this time let us delete.

In [10]:
del androidApps[10473]

In [11]:
for androidApp in androidApps: 
    if len(androidApp) != len(androidHeaders):
        print(androidApp, len(androidApp))
        print(androidApps.index(androidApp))

Nothing is printed now as the erroneous data has been deleted. Let us also quickly check the same for iOS. Lets make this as a function.

In [12]:
def match_header_and_data(headers, apps_data, print_headers=True):
    '''
        Function to check if rows have the same elements as the headers
        
        Parameters
            headers - list of headers in the data set
            apps_data - list of list of data
            print_headers - print headers and length of header, default True
            
        Returns
            None
    '''
    if print_headers:
        print(headers, len(headers))
    for app in apps_data: 
        if len(app) != len(headers):
            print(app, len(app))
            print(apps_data.index(app))

In [13]:
match_header_and_data(androidHeaders, androidApps)
match_header_and_data(iosHeaders, iosApps)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 13
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'] 16


So now both the Android as well as the iOS data looks fine. However since we went through the discussion section for Android, lets also do the [same for iOS](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/discussion) and see if there are errors there.

It is always good to go through the documentation. Also, what they say is generally true, that a data scientist job comprises 80% of data cleaning and only about 20% analyzing.

We see that there are no such discussions for iOS data. Let us move to the next step.

## Cleaning the data - Removing Duplicates

If we observe the headers, then we can know which attributes to check for duplicates.
* App at index 0 in android data
* id at index 0 in ios data

In [14]:
def find_duplicates(apps_data):
    '''
        Function to find duplicates in the data
        
        Parameter:
            apps_data - list of lists of data
        
        Returns:
            2 separate list of duplicate and unique data
    '''
    duplicate_apps = []
    unique_apps = []
    for app in apps_data:
        if app[0] in unique_apps:
            duplicate_apps.append(app[0])
        else:
            unique_apps.append(app[0])
    return duplicate_apps, unique_apps

In [15]:
duplicate_apps_ios, unique_apps_ios = find_duplicates(iosApps[1:])
print(len(duplicate_apps_ios))

0


Nice! There are no duplicates in ios data. Let us check for android now.

In [16]:
duplicate_apps_android, unique_apps_android = find_duplicates(androidApps[1:])
print(len(duplicate_apps_android))

1181


Ok, so there are 1181 duplicates in the android data. Some examples:

In [17]:
print(duplicate_apps_android[:20])

['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software', 'MailChimp - Email, Marketing Automation', 'Crew - Free Messaging and Scheduling', 'Asana: organize team projects', 'Google Analytics', 'AdWords Express']


So we definitely need to remove all the duplicates and have only one entry for the respective apps for our analysis. We can remove randomly but lets us evaluate a few duplicates further and see if we can come up with a better strategy to remove duplicates.

In [18]:
print(androidHeaders)
for app in androidApps[1:]:
    if app[0] == "Slack": # Slack may have duplicates
        print(app)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51507', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']
['Slack', 'BUSINESS', '4.4', '51510', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'August 2, 2018', 'Varies with device', 'Varies with device']


So we see Slack has 3 entries and what varies is the number of reviews. Maybe we can pick the entry that has the highest value as the higher the number of reviews, more reliabe the rating would be.

To remove these duplicates
* Create a dictionary, where each dictionary key is a unique app name and the corresponding dictionary value is the highest number of reviews of that app.
* Use the information stored in the dictionary and create a new dataset, which will have only one entry per app (and for each app, we'll only select the entry with the highest number of reviews).

In [19]:
reviews_max = {}
for app in androidApps[1:]:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    elif name not in reviews_max:
        reviews_max[name] = n_reviews


print(len(reviews_max))
print(reviews_max["Slack"]) # Earlier we had checked it should be 51510

9659
51510.0


In [20]:
cleanedAndroidApps = []
alreadyAdded = []
for app in androidApps[1:]:
    name = app [0]
    num_reviews = float(app[3])
    max_reviews = reviews_max[name]
    if name not in alreadyAdded and num_reviews >= max_reviews:
        cleanedAndroidApps.append(app)
        alreadyAdded.append(name)
        
explore_data(cleanedAndroidApps, 0, 5, True) # To see the cleaned data and see if rows match our calculation or not

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 

['Pixel Draw - Number Art Coloring Book', 'ART_AND_DESIGN', '4.3', '967', '2.8M', '100,000+', 'Free', '0', 'Everyone', 'Art & Design;Creativity', 'June 20, 2018', '1.1', '4.4 and up'] 

['Paper flowers instructions', 'ART_AND_DESIGN', '4.4', '167', '5.6M', '50,000+', 'Free', '0', 'Everyone', 'Art & Design', 'March 26, 2017', '1.0', '2.3 and up'] 

Number of rows :  9659
Number of columns :  13


Now that we have removed the duplicates let us move to removing non English apps.

## Cleaning the data - Removing Non-English Apps

If an app name contains a character that is greater that is not in the ASCII representation of english characters, then it probably means that the app has a non-English name. 

In [21]:
def is_english(text):
    '''
        Function to check if a text is english or not
        
        Parameter:
            text - string 
        
        Returns:
            true or false
    '''
    for character in text:
        if ord(character) > 127:
            return False
    return True

In [22]:
print(is_english("Instagram"))
print(is_english("爱奇艺PPS -《欢乐颂2》电视剧热播"))
print(is_english("Docs To Go™ Free Office Suite"))
print(is_english("Instachat 😜"))

True
False
False
False


As we can see if we do a comparison like this then a lot of apps with English name but with some other characters or emoticons will be rejected from our analysis. To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [23]:
def is_english_app(text):
    '''
        Function to check if a text is english or not
        
        Parameter:
            text - string 
        
        Returns:
            true or false
    '''
    count = 0
    for character in text:
        if ord(character) > 127:
            count += 1
            if count > 3:
                return False
    return True

In [24]:
print(is_english_app("Instagram"))
print(is_english_app("爱奇艺PPS -《欢乐颂2》电视剧热播"))
print(is_english_app("Docs To Go™ Free Office Suite"))
print(is_english_app("Instachat 😜"))

True
False
True
True


Ok, this seems to be fairly ok. Let us use this function to filter out non English apps.

In [25]:
cleaned_english_android_apps = []
for app in cleanedAndroidApps:
    name = app[0]
    if is_english_app(name):
        cleaned_english_android_apps.append(app)

print(len(cleaned_english_android_apps))

cleaned_english_ios_apps = []
for app in iosApps[1:]:
    name = app[1]
    if is_english_app(name):
        cleaned_english_ios_apps.append(app)

print(len(cleaned_english_ios_apps))        


9614
6183


Nice! We see that now there are 9614 English Android apps and 6183 English IOS apps.

## Isolating Free Apps

We only need to analyse for free apps. Let us isolate that.

Check the headers to identify the pricing column.

In [26]:
print(iosHeaders)
explore_data(cleaned_english_ios_apps, 0, 3, False)

print(androidHeaders)
explore_data(cleaned_english_android_apps,0, 3, False)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'] 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1'] 

['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1'] 

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U

* For IOS, column 4, 'price'
* For Android, column 7, 'Price'

We can filter based on the above 2 columns.

In [27]:
free_apps_android = []
for app in cleaned_english_android_apps:
    price = app[7]
    if price == "0":
        free_apps_android.append(app)
print(len(free_apps_android))

8864


In [28]:
free_apps_ios = []
for app in cleaned_english_ios_apps:
    price = app[4]
    if price == "0.0":
        free_apps_ios.append(app)
print(len(free_apps_ios))

3222


We see we have 8864 free, english android apps and 3222 free english ios apps.

## Recap

Let us recap once. Till now 

* We removed inaccurate data
* We removed duplicate data
* We removed non English apps
* We isolated free apps

Great. Our goal was to find out what kind of apps attract more users - come up with suggestions on what apps to build which can be profitable on the play store and the app store.

## Let us start with finding most common genres.

In [29]:
print("Android headers and 3 entries")
print(androidHeaders)
explore_data(free_apps_android, 0, 3, False)

print("\n IOS headers and 3 entries")
print(iosHeaders)
explore_data(free_apps_ios, 0, 3, False)

Android headers and 3 entries
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'] 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up'] 

['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up'] 


 IOS headers and 3 entries
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['2

As we see in the data, the column "prime_genre" in ios data and "Genres" and "Category" in android data should help us determine common genres.

In [30]:
def freq_table(dataset, index):
    '''
        Function to create a frequency table
        
        Parameter:
            dataset - list
            index - column on which frequency is to be created
        
        Returns:
            frequency table with percentage frequencies
    '''
    frequency_table = {}
    total = len(dataset)
    for data in dataset:
        value = data[index]
        if value in frequency_table:
            frequency_table[value] += 1
        else:
            frequency_table[value] = 1
    frequency_percentages = {}
    for key in frequency_table:
        frequency_percentages[key] = float(frequency_table[key] / total) * 100
    return frequency_percentages

In [31]:
def display_table(dataset, index):
    """
        A long approach to sort dict - there are much better
        alternatives available
    """
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

Let us generate frequency tables for android apps based on the column "Genres" (column 9) and "Category" (column 1) and analyse the data.

In [32]:
display_table(free_apps_android, 1)  # By Category

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [33]:
display_table(free_apps_android, 9)  # By Genre

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

Let us do the same for IOS - prime_genre column 11

In [34]:
display_table(free_apps_ios, 11)  # By prime genre

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


### IOS Findings

* Most common genre - Games - greater than 58%
* Second most common - Entertainment - almost 8%
* Mostly apps which are designed for entertainment purpose are more as compared to that designed for practical purpose.

### Android Findings

* Most common genres - Family, Games, Tool, Productivity
* Genres and category - difference not very clear though looks like genres are more granular
* A little healthier mix of apps designed for entertainment as well as practical purposes

Note - since we are looking at a big picture view, we can ignore genre for now and focus on Category.

## Most popular apps by genres - IOS

The frequency tables we analyzed on the previous screen showed us that apps designed for fun dominate the App Store, while Google Play shows a more balanced landscape of both practical and fun apps. Now, we'd like to determine the kind of apps with the most users.

One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the **Installs** column (column 5), but this information is missing for the App Store data set. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the **rating_count_tot** (column 5) app.

In [37]:
prime_genre_freq_table = freq_table(free_apps_ios, 11)

In [41]:
for genre in prime_genre_freq_table:
        total = 0
        len_genre = 0
        for app in free_apps_ios:
            genre_app = app[11]
            if genre == genre_app:
                rating_count = float(app[5])
                total += rating_count
                len_genre += 1
        print(genre, total/len_genre)
                

Social Networking 71548.34905660378
Photo & Video 28441.54375
Games 22788.6696905016
Music 57326.530303030304
Reference 74942.11111111111
Health & Fitness 23298.015384615384
Weather 52279.892857142855
Utilities 18684.456790123455
Travel 28243.8
Shopping 26919.690476190477
News 21248.023255813954
Navigation 86090.33333333333
Lifestyle 16485.764705882353
Entertainment 14029.830708661417
Food & Drink 33333.92307692308
Sports 23008.898550724636
Book 39758.5
Finance 31467.944444444445
Education 7003.983050847458
Productivity 21028.410714285714
Business 7491.117647058823
Catalogs 4004.0
Medical 612.0


Highest avg rating is for navigation and social networking. Let us deep dive before coming to any conclusions.

In [45]:
for app in free_apps_ios:
    if app[11] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Interesting, because of Google and Waze this figure is skewed. Let us check for social networking.

In [46]:
for app in free_apps_ios:
    if app[11] == 'Social Networking':
        print(app[1], ':', app[5]) # print name and number of ratings

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

Once again, the bigger companies/apps, dominate - Facebook, Pinterest etc and smaller companies apps or Indie apps may not show up. The avg number of ratings is skewed by these apps which have a lot of installs. We can do an analysis rmeoving these apps but that is for another day. 

Similalry, Reference apps have 74,942 user ratings on avg, but  Bible and Dictionary.com skew up the average rating.

In [50]:
for app in free_apps_ios:
    if app[11] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


From [DQ solution](https://github.com/dataquestio/solutions/blob/master/Mission350Solutions.ipynb)

However, this niche seems to show some potential. One thing we could do is take another popular book and turn it into an app where we could add different features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes about the book, etc. On top of that, we could also embed a dictionary within the app, so users don't need to exit our app to look up words in an external app.

This idea seems to fit well with the fact that the App Store is dominated by for-fun apps. This suggests the market might be a bit saturated with for-fun apps, which means a practical app might have more of a chance to stand out among the huge number of apps on the App Store.

Other genres that seem popular include weather, book, food and drink, or finance. The book genre seem to overlap a bit with the app idea we described above, but the other genres don't seem too interesting to us:

* Weather apps — people generally don't spend too much time in-app, and the chances of making profit from in-app adds are low. Also, getting reliable live weather data may require us to connect our apps to non-free APIs.
* Food and drink — examples here include Starbucks, Dunkin' Donuts, McDonald's, etc. So making a popular food and drink app requires actual cooking and a delivery service, which is outside the scope of our company.
* Finance apps — these apps involve banking, paying bills, money transfer, etc. Building a finance app requires domain knowledge, and we don't want to hire a finance expert just to build an app.


## Most popular apps by genres - Android

In [51]:
display_table(free_apps_android, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


While these are not exact numbers, for this study we can use these at their face value - just convert properly to numbers.

In [59]:
category_freq_table = freq_table(free_apps_android, 1)

for category in category_freq_table:
        total = 0
        len_category = 0
        for app in free_apps_android:
            category_app = app[1]
            if category == category_app:
                rating_count = float(app[5].replace(",","").replace("+",""))
                total += rating_count
                len_category += 1
        print(category, " : ", total/len_category)

ART_AND_DESIGN  :  1986335.0877192982
AUTO_AND_VEHICLES  :  647317.8170731707
BEAUTY  :  513151.88679245283
BOOKS_AND_REFERENCE  :  8767811.894736841
BUSINESS  :  1712290.1474201474
COMICS  :  817657.2727272727
COMMUNICATION  :  38456119.167247385
DATING  :  854028.8303030303
EDUCATION  :  1833495.145631068
ENTERTAINMENT  :  11640705.88235294
EVENTS  :  253542.22222222222
FINANCE  :  1387692.475609756
FOOD_AND_DRINK  :  1924897.7363636363
HEALTH_AND_FITNESS  :  4188821.9853479853
HOUSE_AND_HOME  :  1331540.5616438356
LIBRARIES_AND_DEMO  :  638503.734939759
LIFESTYLE  :  1437816.2687861272
GAME  :  15588015.603248259
FAMILY  :  3695641.8198090694
MEDICAL  :  120550.61980830671
SOCIAL  :  23253652.127118643
SHOPPING  :  7036877.311557789
PHOTOGRAPHY  :  17840110.40229885
SPORTS  :  3638640.1428571427
TRAVEL_AND_LOCAL  :  13984077.710144928
TOOLS  :  10801391.298666667
PERSONALIZATION  :  5201482.6122448975
PRODUCTIVITY  :  16787331.344927534
PARENTING  :  542603.6206896552
WEATHER  :  50

From [DQ solution](https://github.com/dataquestio/solutions/blob/master/Mission350Solutions.ipynb)

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs.

In [60]:
for app in free_apps_android:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

If we removed all the communication apps that have over 100 million installs, the average would be reduced roughly ten times

In [61]:
under_100_m = []

for app in free_apps_android:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

From DQ

We see the same pattern for the video players category, which is the runner-up with 24,727,872 installs. The market is dominated by apps like Youtube, Google Play Movies & TV, or MX Player. The pattern is repeated for social apps (where we have giants like Facebook, Instagram, Google+, etc.), photography apps (Google Photos and other popular photo editors), or productivity apps (Microsoft Word, Dropbox, Google Calendar, Evernote, etc.).

Again, the main concern is that these app genres might seem more popular than they really are. Moreover, these niches seem to be dominated by a few giants who are hard to compete against.

The game genre seems pretty popular, but previously we found out this part of the market seems a bit saturated, so we'd like to come up with a different app recommendation if possible.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

In [62]:
for app in free_apps_android:
    if app[1] == 'BOOKS_AND_REFERENCE':
        print(app[0], ':', app[5])

E-Book Read - Read Book for free : 50,000+
Download free book with green book : 100,000+
Wikipedia : 10,000,000+
Cool Reader : 10,000,000+
Free Panda Radio Music : 100,000+
Book store : 1,000,000+
FBReader: Favorite Book Reader : 10,000,000+
English Grammar Complete Handbook : 500,000+
Free Books - Spirit Fanfiction and Stories : 1,000,000+
Google Play Books : 1,000,000,000+
AlReader -any text book reader : 5,000,000+
Offline English Dictionary : 100,000+
Offline: English to Tagalog Dictionary : 500,000+
FamilySearch Tree : 1,000,000+
Cloud of Books : 1,000,000+
Recipes of Prophetic Medicine for free : 500,000+
ReadEra – free ebook reader : 1,000,000+
Anonymous caller detection : 10,000+
Ebook Reader : 5,000,000+
Litnet - E-books : 100,000+
Read books online : 5,000,000+
English to Urdu Dictionary : 500,000+
eBoox: book reader fb2 epub zip : 1,000,000+
English Persian Dictionary : 500,000+
Flybook : 500,000+
All Maths Formulas : 1,000,000+
Ancestry : 5,000,000+
HTC Help : 10,000,000+
E

Refer - https://github.com/dataquestio/solutions/blob/master/Mission350Solutions.ipynb

## Conclusion

We can start with taking a popular (recent) book and build interesting features on top of that - daily quotes, audio format, quizzes, community forum and so on.